In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('flipitnews-data.csv')

In [3]:
df.head()

,Category,Article
0,Technology,tv future in the hands of viewers with home th...
1,Business,worldcom boss left books alone former worldc...
2,Sports,tigers wary of farrell gamble leicester say ...
3,Sports,yeading face newcastle in fa cup premiership s...
4,Entertainment,ocean s twelve raids box office ocean s twelve...


In [4]:
df.shape

(2225, 2)

In [5]:
words_num = [len(df['Article'][x].split()) for x in range(len(df['Article']))]

In [6]:
df_words = pd.DataFrame(words_num, columns=['words_num'])

In [7]:
df_words.describe()

,words_num
count,2225.000000
mean,390.295281
std,241.753128
min,90.000000
25%,250.000000
50%,337.000000
75%,479.000000
max,4492.000000


In [8]:
len(df['Article'][5].split())

633

In [9]:
train_text, test_text, train_labels, test_labels = train_test_split(df['Article'], df['Category'], test_size=0.2, random_state=42)

In [10]:
len(train_text.to_list())

1780

In [11]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# tokenize a set of texts
train_encodings = tokenizer(train_text.tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_text.tolist(), truncation=True, padding=True, max_length=512)


c:\Users\HP\Documents\GitHub\flipit_news_business_case\myenv2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
len(train_encodings['input_ids'][2])

512

In [13]:
# convert the lists to tensors
import torch

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_labels = train_labels.astype('category').cat.codes
test_labels = test_labels.astype('category').cat.codes   
train_dataset = NewsDataset(train_encodings, train_labels.to_list())
test_dataset = NewsDataset(test_encodings, test_labels.to_list())

In [14]:
train_dataset[0].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [15]:
len(set(train_labels))

5

In [16]:
# Load the BERT model
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(train_labels)))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# define the optimizer and trainer
from transformers import AdamW
from torch.utils.data import DataLoader

# create the data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

c:\Users\HP\Documents\GitHub\flipit_news_business_case\myenv2\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


: 

In [ ]:
# fine tune the model
from transformers import get_scheduler
from tqdm import tqdm

# define the learning rate scheduler
num_epochs = 1
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# training loop
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())



  0%|          | 0/28 [00:00<?, ?it/s]

In [ ]:
# evaluate the model
from sklearn.metrics import accuracy_score

model.eval()
predictions = []
real_values = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        predictions.extend(predicted.cpu().numpy())
        real_values.extend(labels.cpu().numpy())

# calculate the accuracy
accuracy = accuracy_score(real_values, predictions)
print(f'Test Accuracy: {accuracy}')


In [ ]:
# save the model
model.save_pretrained('news_model')
tokenizer.save_pretrained('news_model')

In [ ]:
# load the model for inference
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained('news_model')
tokenizer = BertTokenizer.from_pretrained('news_model')

# prepare the input
text = 'The stock market is going up'
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
outputs = model(**inputs)
_, predicted = torch.max(outputs.logits, 1)
print(f'Predicted category: {predicted.item()}')